In [180]:
!pip install sounddevice -q
!pip install ipywebrtc -q
!pip install mido python-rtmidi -q
!sudo apt-get install libportaudio2 -y -qq

In [182]:
import numpy as np
import librosa
import sounddevice as sd
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio
import ipywidgets as widgets
from google.colab import output
import random
import math
import mido
import rtmidi

In [224]:
def get_leading_sample(input_array,T_start,FILE_SAMPLING_RATE,TRAIN_SAMPLING_RATE,WINDOW_SIZE):
  t=math.floor(T_start*FILE_SAMPLING_RATE)
  if t < WINDOW_SIZE*FILE_SAMPLING_RATE:
    return(np.concat((np.zeros((WINDOW_SIZE*FILE_SAMPLING_RATE - t, 2)),input_array[:t,]),axis=0)[::math.floor(FILE_SAMPLING_RATE/TRAIN_SAMPLING_RATE),])
  else:
    return(input_array[(t-WINDOW_SIZE*FILE_SAMPLING_RATE):t,][::math.floor(FILE_SAMPLING_RATE/TRAIN_SAMPLING_RATE),])
def get_pitch(leading_sample, velocity=63):
  return(random.randint(32, 96))
def get_event(input_array,T_start,T_end,FILE_SAMPLING_RATE,TRAIN_SAMPLING_RATE,WINDOW_SIZE):
  pitch=get_pitch(get_leading_sample(input_array,T_start,FILE_SAMPLING_RATE,TRAIN_SAMPLING_RATE,WINDOW_SIZE))
  return(np.iinfo(np.int16).max * 0.5 * np.sin(2. * np.pi * 440 * (2**((pitch - 69) / 12)) * np.linspace(0., T_end-T_start, int((T_end-T_start) * sr))),
         np.tile([pitch,63],(math.floor(T_end*FILE_SAMPLING_RATE)-math.floor(T_start*FILE_SAMPLING_RATE),1)),
        mido.Message('note_on',
                    note=pitch,
                    velocity=63,
                    time=math.floor(T_start*FILE_SAMPLING_RATE)),
        mido.Message('note_off',
                     note=pitch,
                    velocity=0,
                     time=math.floor(T_end*FILE_SAMPLING_RATE)-1))

In [172]:
#Record Audio Clip
output.enable_custom_widget_manager()
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [225]:
#Process Clip
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

#Extract Onset Events
filename = 'my_recording.wav'
y, sr = librosa.load(filename)
onset_events = np.concat((librosa.onset.onset_detect(y=y, sr=sr, units='time'), np.array([len(y)/sr])))
input_array = np.zeros((math.floor(len(y)),2))
Y = np.zeros_like(y)

#Set Sampling Properties
WINDOW_SIZE = 1
TRAIN_SAMPLING_RATE = 10
FILE_SAMPLING_RATE = sr

#Initialize MidiFile
f = mido.MidiFile(type=1, ticks_per_beat=sr)
ff = mido.MidiTrack()
f.tracks.append(ff)
ff.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(60), time=0))

#Generate Melody
for i in range(len(onset_events)-1):
  T_start=onset_events[i]
  T_end=onset_events[i+1]
  Y[int(T_start * sr):(int(T_start * sr) + int((T_end-T_start) * sr))], \
    input_array[math.floor(T_start*FILE_SAMPLING_RATE):math.floor(T_end*FILE_SAMPLING_RATE),:], \
    m1, m2 = get_event(input_array,T_start,T_end,FILE_SAMPLING_RATE,TRAIN_SAMPLING_RATE,WINDOW_SIZE)
  ff.append(m1)
  ff.append(m2)

In [222]:
#Playback (original)
Audio(y, rate=sr)

In [226]:
#Playback (melody)
Audio(Y, rate=sr)